# * Kaggle Connecting

In [0]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

-rw-r--r-- 1 root root 66 Dec 14 08:51 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=673b96fe57df09c18d9aea6a902686ff8070be8ef3f773578ff52f5aa3512bce
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
Kaggle API 1.5.6


In [0]:
! kaggle competitions download -c 2019-ml-finalproject

 75% 49.0M/65.0M [00:00<00:00, 88.2MB/s]
100% 65.0M/65.0M [00:00<00:00, 126MB/s] 


In [0]:
import zipfile
import os

os.mkdir('/content/input')

zip_ref = zipfile.ZipFile("/content/2019-ml-finalproject.zip", 'r')
zip_ref.extractall("/content/input")
zip_ref.close()

# * opencv version changing

In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 94kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 77kB/s 


# * import library

In [0]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import torch
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook as tqdm

import glob

In [0]:
cv2.__version__

'3.4.2'

# * Feature Extraction
- Dense Sift

In [0]:
# label readying.
df_data = pd.read_csv('/content/input/Label2Names.csv', header=None)

In [0]:
# train dense sift
train_path = "/content/input/train"

train_des = list()
train_images = list()
train_label = list()
train_fea = list()

dense = 8
img_size = 256

for cls in tqdm(os.listdir(train_path)):
  img_list = os.listdir(train_path + '/' + cls)
  img_list.sort()
  if cls == 'BACKGROUND_Google':
    label = 102
    #continue
  else :
    label = (df_data.index[df_data[1]==cls] + 1).tolist()[0]

  for img in img_list:

    image = cv2.imread(train_path + '/' + cls + '/' + img)

    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (img_size, img_size))
    sift = cv2.xfeatures2d.SIFT_create()

    kp = [cv2.KeyPoint(x, y, dense) for y in range(0, gray.shape[0]
                                                   , dense) for x in range(0, gray.shape[1], dense)]
    keypoint, des = sift.compute(gray, kp)

    train_des.append(des)
    train_fea.append(des)
    train_images.append(gray)
    train_label.append(label)

In [0]:
# test dense sift
test_path = "/content/input/testAll_v2"

test_des = list()
test_images = list()
test_fea = list()

dense = 8
img_size = 256

img_list = os.listdir(test_path)
img_list.sort()

for img in tqdm(img_list):
  image = cv2.imread(test_path + '/' + img)

  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (img_size, img_size))
  sift = cv2.xfeatures2d.SIFT_create()

  kp = [cv2.KeyPoint(x, y, dense) for y in range(0, gray.shape[0]
                                                   , dense) for x in range(0, gray.shape[1], dense)]
  keypoint, des = sift.compute(gray, kp)
  
  test_des.append(des)
  test_images.append(gray)
  test_fea.append(des)

In [0]:
# import pickle

# # Save pickle
# with open("densetrain_des.pickle", "wb") as f:
#   pickle.dump(train_des, f)

# with open("densetest_des.pickle", "wb") as f:
#   pickle.dump(test_des, f)


In [0]:
# with open("densetrain_des.pickle", "rb") as r:
#   read_data = pickle.load(r)

# with open("densetest_des.pickle", "rb") as r:
#   read_data = pickle.load(r)

In [0]:
train_images = np.array(train_images)
train_images.shape

(3060, 256, 256)

# * Checking the shape

In [0]:
train_des = np.array(train_des)
train_des.shape

(3133440, 128)

In [0]:
test_des = np.array(test_des)
test_des.shape

(1732608, 128)

In [0]:
# for codebook histogram, reshape 1d, 128 dimensions

train_des = train_des.reshape(-1,128)
train_des.shape

(3133440, 128)

In [0]:
test_des = test_des.reshape(-1,128)
test_des.shape

(1732608, 128)

# Minibatchkmeans

In [0]:
# gpu kmeans install
pip install kmc2

     |████████████████████████████████| 102kB 5.7MB/s 
     |████████████████████████████████| 163kB 34.1MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252222 sha256=15e020704fb0e3eaf96d1496c2246f298f89f4550cc53cc266bfef5ae80115cf
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
import kmc2
import copy
from sklearn.cluster import MiniBatchKMeans

In [0]:
#특징에 대한 클러스터링을 진행
codebook_size = 800
seeding = kmc2.kmc2(train_des, codebook_size)
kmeans = MiniBatchKMeans(codebook_size, init = seeding).fit(train_des)
codebook = kmeans.cluster_centers_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=800. Setting it to 3*k
  init_size=init_size)


# HistogramSPM (2_depth)

In [0]:
import math
from scipy.cluster.vq import vq

def Fast_getImageFeaturesSPM2(L, img_des, kmeans, k):

    W = int(256/8)
    H = int(256/8)      
    h = []
     
    img_des = img_des.reshape(32,32,128)
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):        
                desc = img_des[y:y+h_step, x:x+w_step,:].reshape(-1,128)                
                predict_vq = vq(desc,codebook)
                histo = np.bincount(predict_vq[0], minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    return hist


# get histogram representation for training/testing data
def Fast_getHistogramSPM2(L, data, kmeans, k):    
    x = []
    for i in range(len(data)):        
        hist = Fast_getImageFeaturesSPM2(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [0]:
train_fea = np.array(train_fea)

In [0]:
train_fea.shape

(3060, 1024, 128)

In [0]:
train_hist = Fast_getHistogramSPM2(2,train_fea, kmeans, codebook_size)

In [0]:
test_hist = Fast_getHistogramSPM2(2,test_fea, kmeans, codebook_size)

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
train_scaler = StandardScaler().fit(train_hist)
train_histo = train_scaler.transform(train_hist)

test_histo = train_scaler.transform(test_hist)

In [0]:
# spm kernel

def histogramIntersection(M,N):
  m = M.shape[0]
  n = N.shape[0]
  result = np.zeros ((m,n))
  for i in range(m):
    for j in range(n):
      temp = np.sum(np.minimum(M[i],N[j]))
      result[i][j] = temp
  return result

In [0]:
from sklearn.svm import SVC 

from sklearn.pipeline import make_pipeline

In [0]:
gramMatrix = histogramIntersection(train_histo,train_histo)
predictMatrix = histogramIntersection(test_histo,train_histo)

In [0]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001,0.005,0.01, 0.05, 0.1],
              'gamma': [0.0000001,0.0000005,0.000001]}
grid = GridSearchCV(SVC(kernel = 'precomputed'), param_grid)

%time grid.fit(gramMatrix,train_label)
print(grid.best_params_)

In [0]:
SVMresults=grid.predict(predictMatrix)

In [0]:
SVMresults.shape

(1692,)

In [0]:
img_list = os.listdir("/content/input/testAll_v2")
img_list.sort()

In [0]:
yfit = SVMresults.reshape(-1,1)
result_img_list = np.array(img_list).reshape(-1,1)
total_result = np.hstack([result_img_list,yfit])

In [0]:
df = pd.DataFrame(total_result, columns=["Id","Category"])
df.to_csv('results-yongho-v3.csv',index=False, header=True)

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-yongho-v3.csv -m "Final_Term_Project"

100% 29.6k/29.6k [00:02<00:00, 11.2kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 